In [1]:
!pip install neo4j-graphrag

In [2]:
!pip install openai

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

## ERExtractionTemplate 활용하여 텍스트 안에서 지식 그래프 추출하기

ERExtractionTemplate 은 주어진 텍스트 내에서 엔티티(노드)를 추출하고, 그 엔티티들 간의 관계 또한 추출하여 Knowledge Graph를 구축해주는 기능을 합니다.

[Neo4j GraphRAG API Documentation](https://neo4j.com/docs/neo4j-graphrag-python/current/api.html#erextractiontemplate)

In [4]:
from neo4j_graphrag.generation.prompts import ERExtractionTemplate

prompt_template = ERExtractionTemplate()

prompt = prompt_template.format(
    schema = '',
    text = '''
        입력 텍스트
    ''',
    examples = ''
)
print(prompt)


You are a top-tier algorithm designed for extracting
information in structured formats to build a knowledge graph.

Extract the entities (nodes) and specify their type from the following text.
Also extract the relationships between these nodes.

Return result as JSON using the following format:
{"nodes": [ {"id": "0", "label": "Person", "properties": {"name": "John"} }],
"relationships": [{"type": "KNOWS", "start_node_id": "0", "end_node_id": "1", "properties": {"since": "2024-08-01"} }] }

Use only the following node and relationship types (if provided):


Assign a unique ID (string) to each node, and reuse it to define relationships.
Do respect the source and target node types for relationship and
the relationship direction.

Make sure you adhere to the following rules to produce valid JSON objects:
- Do not return any additional information other than the JSON in it.
- Omit any backticks around the JSON - simply output the JSON on its own.
- The JSON object must not wrapped into a 

LLM에 입력하여 답변 생성하기

In [5]:
from neo4j_graphrag.llm import OpenAILLM

prompt_template = ERExtractionTemplate()
llm = OpenAILLM(model_name="gpt-4o", model_params={"temperature": 0})

input_text = '''
    빈수와 수지는 친구입니다. 빈수는 수지를 좋아합니다.
'''
prompt = prompt_template.format(
    schema = '',
    text = input_text,
    examples = ''
)

response = llm.invoke(prompt)

In [6]:
print(response.content)

{
  "nodes": [
    {"id": "0", "label": "Person", "properties": {"name": "빈수"}},
    {"id": "1", "label": "Person", "properties": {"name": "수지"}}
  ],
  "relationships": [
    {"type": "FRIENDS_WITH", "start_node_id": "0", "end_node_id": "1"},
    {"type": "LIKES", "start_node_id": "0", "end_node_id": "1"}
  ]
}


In [7]:
input_text = '''
    jwg은 10월 10일에 후쿠오카에 다녀왔습니다.
    후쿠오카의 주요 지하철 역인 텐진역과 하카타역 주변에서 주로 시간을 보냈습니다.
    10월 12일에 한국에 돌아왔습니다.
'''
prompt = prompt_template.format(
    schema = '',
    text = input_text,
    examples = ''
)

response = llm.invoke(prompt)
print(response.content)

{
  "nodes": [
    {"id": "0", "label": "Person", "properties": {"name": "jwg"}},
    {"id": "1", "label": "Location", "properties": {"name": "Fukuoka"}},
    {"id": "2", "label": "Location", "properties": {"name": "Tenjin Station"}},
    {"id": "3", "label": "Location", "properties": {"name": "Hakata Station"}},
    {"id": "4", "label": "Location", "properties": {"name": "Korea"}}
  ],
  "relationships": [
    {"type": "VISITED", "start_node_id": "0", "end_node_id": "1", "properties": {"date": "2023-10-10"}},
    {"type": "SPENT_TIME_AT", "start_node_id": "0", "end_node_id": "2", "properties": {}},
    {"type": "SPENT_TIME_AT", "start_node_id": "0", "end_node_id": "3", "properties": {}},
    {"type": "RETURNED_TO", "start_node_id": "0", "end_node_id": "4", "properties": {"date": "2023-10-12"}}
  ]
}


## [응용] 나만의 영화 줄거리 완성하고 등장인물 관계도 만들기

### Step 1. 이야기를 써내려가며 지식그래프 업데이트 하기

In [8]:
import json
import neo4j
from neo4j_graphrag.experimental.components.kg_writer import Neo4jWriter
from neo4j_graphrag.experimental.components.types import Neo4jGraph

graph_json = ''
graph_history = ''
while(True):
    user_input = input("이야기를 들려주세요 🗣️")

    if(user_input == '완성'):
        break
    prompt = prompt_template.format(
        schema = '',
        text = user_input
            + """(Continue extracting the graph for the following Input text.
                Ensure you retain the existing nodes and relationships from the graph history
                and add only new nodes and relationships.\n
                Graph History : """ + graph_history + ")",
        examples = ''
    )
    print(prompt)
    response = llm.invoke(prompt) # 그래프 추출
    print(response.content)
    graph_history = response.content

    # 간혹 답변에 ```json ``` 이 포함되는 경우가 있어 {} 를 기준으로 split
    graph_json = json.loads(graph_history[graph_history.find("{"):graph_history.rfind("}")+1])

이야기를 들려주세요 🗣️서울 변두리의 오래된 재개발 예정 동네 ‘겹동’. 이곳에서 작은 인쇄소를 운영하는 **윤서준(35)**은 7년 전 이유도 모른 채 떠난 연인 **한지은(34)**과 재개발 설명회에서 우연히 다시 마주친다. 지은은 대형 건설사의 도시재생 팀장으로, 서준의 동네를 철거해야 하는 입장이다.  서준은 아직도 지은이 왜 떠났는지 모른다. 하지만 지은은 알고 있다. 그가 가장 신뢰하던 형이자 멘토였던 **윤태준(42)**이, 과거 자신에게 되돌릴 수 없는 선택을 강요했다는 사실을.  태준은 현재 겹동에서 영향력 있는 구의원이자 재개발 추진의 핵심 인물이다. 겉으로는 동생을 위하는 척하지만, 실상은 모든 관계를 자기 합리화로 엮어온 인물이다.  한편, 서준의 인쇄소에서 일하는 **박소연(29)**은 지은의 대학 후배이자, 태준의 비밀 연인이었다. 소연은 지은이 겹동에 온 진짜 이유가 재개발이 아니라는 걸 어렴풋이 눈치챈다. 지은은 사실, 태준의 과거 비리를 폭로하기 위해 이 동네로 돌아온 것이다.  여기에 겹동에서 30년째 분식집을 운영하는 **최말순(68)**이 있다. 말순은 태준이 지금의 자리에 오를 수 있도록 결정적인 도움을 준 인물이지만, 그 대가로 아들을 잃었다. 그리고 그 사실을 아는 사람은 지은뿐이다.  사람들은 서로를 돕는다고 믿지만, 실은 누군가의 침묵 위에 서 있고 누군가의 오해로 관계를 유지하고 있다.  서준은 진실을 알수록 형을 미워해야 하는지, 사랑을 다시 붙잡아야 하는지, 아니면 아무것도 선택하지 말아야 하는지 흔들린다.  지은은 서준을 여전히 사랑하지만, 자신의 진실이 그를 무너뜨릴 걸 알기에 끝까지 말하지 않으려 한다.  그리고 소연은 누군가의 비밀을 지켜주던 사람이 어느 순간 가장 잔인한 배신자가 될 수 있다는 걸 증명하게 된다.

You are a top-tier algorithm designed for extracting
information in structured formats to build a knowledg

In [9]:
print(graph_history)

{
  "nodes": [
    {"id": "0", "label": "Person", "properties": {"name": "윤서준", "age": "35"}},
    {"id": "1", "label": "Person", "properties": {"name": "한지은", "age": "34"}},
    {"id": "2", "label": "Person", "properties": {"name": "윤태준", "age": "42"}},
    {"id": "3", "label": "Person", "properties": {"name": "박소연", "age": "29"}},
    {"id": "4", "label": "Person", "properties": {"name": "최말순", "age": "68"}},
    {"id": "5", "label": "Place", "properties": {"name": "겹동"}}
  ],
  "relationships": [
    {"type": "KNOWS", "start_node_id": "0", "end_node_id": "1", "properties": {"since": "7 years ago"}},
    {"type": "WORKS_AT", "start_node_id": "0", "end_node_id": "5", "properties": {"role": "인쇄소 운영자"}},
    {"type": "WORKS_AT", "start_node_id": "1", "end_node_id": "5", "properties": {"role": "도시재생 팀장"}},
    {"type": "SIBLING", "start_node_id": "0", "end_node_id": "2"},
    {"type": "MENTOR", "start_node_id": "2", "end_node_id": "0"},
    {"type": "WORKS_AT", "start_node_id": "2", "end

### Step 2. 완성 시 Neo4j 그래프에 담아 한눈에 확인하기


#### 여기서 지식 그래프 확인
https://sandbox.neo4j.com/

In [11]:
graph_json = ''
graph_history = ''

while(True):
    user_input = input("이야기를 들려주세요 🗣️")

    if(user_input == '완성'):
        with neo4j.GraphDatabase.driver("bolt://...", auth=("neo4j", "...")) as driver:
            writer = Neo4jWriter(driver)
            graph = Neo4jGraph(
                nodes= graph_json['nodes'],
                relationships= graph_json['relationships']
            )
            await writer.run(graph)
            break
    prompt = prompt_template.format(
        schema = '',
        text = user_input
            + """(Continue extracting the graph for the following Input text.
                Ensure you retain the existing nodes and relationships from the graph history
                and add only new nodes and relationships.\n
                Graph History : """ + graph_history + ")",
        examples = ''
    )
    print(prompt)
    response = llm.invoke(prompt) # 그래프 추출
    print(response.content)
    graph_history = response.content

    # 간혹 답변에 ```json ``` 이 포함되는 경우가 있어 {} 를 기준으로 split
    graph_json = json.loads(graph_history[graph_history.find("{"):graph_history.rfind("}")+1])

이야기를 들려주세요 🗣️서울 변두리의 오래된 재개발 예정 동네 ‘겹동’. 이곳에서 작은 인쇄소를 운영하는 **윤서준(35)**은 7년 전 이유도 모른 채 떠난 연인 **한지은(34)**과 재개발 설명회에서 우연히 다시 마주친다. 지은은 대형 건설사의 도시재생 팀장으로, 서준의 동네를 철거해야 하는 입장이다.  서준은 아직도 지은이 왜 떠났는지 모른다. 하지만 지은은 알고 있다. 그가 가장 신뢰하던 형이자 멘토였던 **윤태준(42)**이, 과거 자신에게 되돌릴 수 없는 선택을 강요했다는 사실을.  태준은 현재 겹동에서 영향력 있는 구의원이자 재개발 추진의 핵심 인물이다. 겉으로는 동생을 위하는 척하지만, 실상은 모든 관계를 자기 합리화로 엮어온 인물이다.  한편, 서준의 인쇄소에서 일하는 **박소연(29)**은 지은의 대학 후배이자, 태준의 비밀 연인이었다. 소연은 지은이 겹동에 온 진짜 이유가 재개발이 아니라는 걸 어렴풋이 눈치챈다. 지은은 사실, 태준의 과거 비리를 폭로하기 위해 이 동네로 돌아온 것이다.  여기에 겹동에서 30년째 분식집을 운영하는 **최말순(68)**이 있다. 말순은 태준이 지금의 자리에 오를 수 있도록 결정적인 도움을 준 인물이지만, 그 대가로 아들을 잃었다. 그리고 그 사실을 아는 사람은 지은뿐이다.  사람들은 서로를 돕는다고 믿지만, 실은 누군가의 침묵 위에 서 있고 누군가의 오해로 관계를 유지하고 있다.  서준은 진실을 알수록 형을 미워해야 하는지, 사랑을 다시 붙잡아야 하는지, 아니면 아무것도 선택하지 말아야 하는지 흔들린다.  지은은 서준을 여전히 사랑하지만, 자신의 진실이 그를 무너뜨릴 걸 알기에 끝까지 말하지 않으려 한다.  그리고 소연은 누군가의 비밀을 지켜주던 사람이 어느 순간 가장 잔인한 배신자가 될 수 있다는 걸 증명하게 된다.

You are a top-tier algorithm designed for extracting
information in structured formats to build a knowledg